In [2]:
import pandas as pd
import os
os.chdir("/Users/anorawu/Documents/GitHub/CloudSeeding/data/炮台数据")
df = pd.read_csv('df_merge_nodup_coord.csv')

In [3]:
df

,Unnamed: 0,weibo_text,operation_year,operation_month,operation_day,prov,city,county,area,location,coordinate,coordinate_level,address,formatted_address
0,0,勐海县抓住有利天气条件 开展人工增雨作业有效缓解旱情记者黎智仙报道： 3月24日...,2020,3,20,云南省,西双版纳傣族自治州,勐海县,星火山、布朗山、勐混、曼稿保护区、勐邦水库,云南省西双版纳傣族自治州星火山,"100.179358,22.092365",住宅区,西双版纳傣族自治州星火山,云南省西双版纳傣族自治州勐海县星火山
1,1,勐海县抓住有利天气条件 开展人工增雨作业有效缓解旱情记者黎智仙报道： 3月24日...,2020,3,20,云南省,西双版纳傣族自治州,勐海县,星火山、布朗山、勐混、曼稿保护区、勐邦水库,云南省西双版纳傣族自治州布朗山,"100.406481,21.626384",村庄,西双版纳傣族自治州布朗山,云南省西双版纳傣族自治州勐海县布朗山
2,2,勐海县抓住有利天气条件 开展人工增雨作业有效缓解旱情记者黎智仙报道： 3月24日...,2020,3,20,云南省,西双版纳傣族自治州,勐海县,星火山、布朗山、勐混、曼稿保护区、勐邦水库,云南省西双版纳傣族自治州勐混,"100.390518,21.841447",乡镇,西双版纳傣族自治州勐混,云南省西双版纳傣族自治州勐海县勐混镇
3,3,勐海县抓住有利天气条件 开展人工增雨作业有效缓解旱情记者黎智仙报道： 3月24日...,2020,3,20,云南省,西双版纳傣族自治州,勐海县,星火山、布朗山、勐混、曼稿保护区、勐邦水库,云南省西双版纳傣族自治州曼稿保护区,"100.797002,22.009037",市,西双版纳傣族自治州曼稿保护区,云南省西双版纳傣族自治州
4,4,勐海县抓住有利天气条件 开展人工增雨作业有效缓解旱情记者黎智仙报道： 3月24日...,2020,3,20,云南省,西双版纳傣族自治州,勐海县,星火山、布朗山、勐混、曼稿保护区、勐邦水库,云南省西双版纳傣族自治州勐邦水库,"100.290934,21.900714",住宅区,西双版纳傣族自治州勐邦水库,云南省西双版纳傣族自治州勐海县勐邦水库
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9608,9608,#天气实况#【德州市气象台】强对流天气实况：5月19日20时至20日6时我市北部县（市、区）...,2023,5,19,山东省,德州市,NaN,22个作业点,山东省德州市22,"116.359244,37.436492",市,德州市22,山东省德州市
9609,9609,风雨正在赶往五莲路上 今夜到明天将实施人工增雨 我县小樱桃正陆续进入盛花期，一阵...,2022,3,16,山东省,日照市,五莲县,16日夜间到17日上午实施人工增雨作业,山东省日照市16日夜间到17日上午实施人工增雨作业,Not Found,Not Found,日照市16日夜间到17日上午实施人工增雨作业,Not Found
9610,9610,风雨正在赶往五莲路上 今夜到明天将实施人工增雨 我县小樱桃正陆续进入盛花期，一阵...,2022,3,16,山东省,日照市,五莲县,16日夜间到17日上午实施人工增雨作业,山东省返回的具体位置信息为,Not Found,Not Found,返回的具体位置信息为,Not Found
9611,9611,风雨正在赶往五莲路上 今夜到明天将实施人工增雨 我县小樱桃正陆续进入盛花期，一阵...,2022,3,16,山东省,日照市,五莲县,16日夜间到17日上午实施人工增雨作业,山东省日照市,"119.526850,35.416912",市,日照市,山东省日照市


In [4]:
df[df['coordinate_level']=='兴趣点'].count()

Unnamed: 0           361
weibo_text           361
operation_year       361
operation_month      361
operation_day        361
prov                 361
city                 361
county               264
area                 358
location             361
coordinate           361
coordinate_level     361
address              361
formatted_address    361
dtype: int64

In [5]:
df[df['coordinate_level']=='住宅区'].count()

Unnamed: 0           669
weibo_text           669
operation_year       669
operation_month      669
operation_day        669
prov                 669
city                 666
county               587
area                 659
location             669
coordinate           669
coordinate_level     669
address              669
formatted_address    669
dtype: int64

In [6]:
df[df['coordinate_level']=='村庄'].count()

Unnamed: 0           955
weibo_text           955
operation_year       955
operation_month      955
operation_day        955
prov                 955
city                 953
county               847
area                 944
location             955
coordinate           955
coordinate_level     955
address              936
formatted_address    955
dtype: int64

In [7]:
df[df['coordinate_level']=='乡镇'].count()

Unnamed: 0           1385
weibo_text           1385
operation_year       1385
operation_month      1385
operation_day        1385
prov                 1385
city                 1384
county               1237
area                 1368
location             1385
coordinate           1385
coordinate_level     1385
address              1385
formatted_address    1385
dtype: int64